![](ubc_header.png)

# Climate Friendly Food System (CFFS) Labelling Pilot
***

## Objective

Implement the Climate-Friendly and Just Food System (CFFS &Just) definition at the UBC Campus by producing the weighted metric that informs the choice of icon for each menu item.

## Data Source

**Recipes Data**: extracted from UBC culinary services management platform Optimum Control
    
    Items.xml: Raw ingredient Items
    Ingredients.xml: Ingredients/Preps that goes into prep/products recipes
    Preps.xml: Preps that go into final product recipes
    Products.xml: Final Recipes/Menu Items
    Conversions.xml: Unit Conversion Information

**cool_food_ghg.csv**: extracted from the [Cool Food Calculator](https://coolfood.org/pledge/). For this project, we use emission factors for the active total supply chain emissions in the North America region. The food category is at the detailed level for higher results accuracy.

**standard_conversions.csv**: standard unit conversion information

## Assumptions

* The same GHG emission factor will be assigned to different forms (puree, sliced, chopped etc) of the same raw ingredient

* We assume the GHG factors for different varieties of one ingredient are the same (i.e Red/Yellow Onion)

* The loss/add of ingredient weight during the cooking process will be ignored

* The amount of unusable part and wastes during the ingredients processing process will be ignored 

* Ignore GHG emissions from the cooking process

* Assume the GHG emission factor for water is zero, and ignore the water use in the cooking process

* We exclude the GHG emission from sauces/dressing that has no dominant ingredients

## Import Libraries

In [1]:
#!pip3 install pdpipe

In [2]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os

import csv
from itertools import islice
from decimal import Decimal

In [3]:
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse

In [4]:
import openpyxl

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

## Load Data

#### Data Path

In [6]:
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "oc", "Open Kitchen","*.oc"))

In [7]:
filepath_list

['/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06182021_0938.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06232021_0918.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06182021_0918.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06232021_1141.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06182021_1001.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06232021_1155.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06182021_0927.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06232021_0956.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06232021_1202.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_06232021_1111.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/Open Kitchen/IPR_Export_0618202

#### Load Items List

In [8]:
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    path = filepath + '/items.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for item in xtree.iterfind('Item'):
        ItemId.append(item.attrib['id'])
        Description.append(item.findtext('Description'))
        CaseQty.append(item.findtext('CaseQty'))
        CaseUOM.append(item.findtext('CaseUOM'))
        PakQty.append(item.findtext('PakQty'))
        PakUOM.append(item.findtext('PakUOM'))
        InventoryGroup.append(item.findtext('InventoryGroup'))

        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    ).drop_duplicates()

Items.reset_index(drop=True, inplace=True)

In [9]:
Items

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-4271,APPLES GRANNY SMITH,113.000,ea,1.000,CT,PRODUCE
1,I-4971,ARTICHOKE 1/4 SALAD CUT TFC,6.000,LG CAN,2.500,Kg,PRODUCE
2,I-2305,BACON PANCETTA,1.000,Kg,1.000,Kg,MEAT
3,I-1207,BAGUETTE FRENCH,24.000,each,1.000,CT,BREAD
4,I-17203,BALSAMIC GLAZE,2.000,bottle,2.000,L,FOOD - GROCERY
5,I-3141,BASE VEG CONC LIQ G/FREE,4.000,pak,946.000,ml,FOOD - GROCERY
6,I-3619,BAY LEAF WHL SHAKER TFC,8.000,each,84.000,g,SPICES
7,I-59287,BEEF GRD MUSH/ONION BLEND,5.000,Kg,1.000,Kg,MEAT
8,I-4521,BEETS BC,25.000,lb,1.000,lb,PRODUCE
9,I-13050,BEETS GOLDEN UBC,1.000,lb,1.000,lb,PRODUCE


In [10]:
Items.shape

(579, 7)

In [11]:
Items.dtypes

ItemId            object
Description       object
CaseQty           object
CaseUOM           object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [12]:
path = os.path.join(os.getcwd(), "data", "processed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

#### Load Ingredient List

In [13]:
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

for filepath in filepath_list:
    path = filepath + '/Ingredients.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for x in xtree.iterfind('Ingredient'):
        IngredientId.append(x.attrib['ingredient'])
        Conversion.append(x.attrib['conversion'])
        InvFactor.append(x.attrib['invFactor'])
        Qty.append(x.attrib['qty'])
        Recipe.append(x.attrib['recipe'])
        Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()

Ingredients.reset_index(drop=True, inplace=True)

In [14]:
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-18746,1.000,Kg,1.00000000,1.0000,P-10241
1,I-3388,1.000,L,1.00000000,0.3058,P-10496
2,I-4660,2.270,Kg,2.20462000,0.6942,P-10496
3,I-3451,2.560,L,1.00000000,1.2800,P-13933
4,I-4679,1.000,BUNCH,1.00000000,0.0063,P-18318


In [15]:
Ingredients.shape

(5240, 6)

In [16]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [17]:
path = os.path.join(os.getcwd(), "data", "processed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

#### Load Preps List

In [18]:
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    path = filepath + '/Preps.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for x in xtree.iterfind('Prep'):
        PrepId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        PakQty.append(x.findtext('PakQty'))
        PakUOM.append(x.findtext('PakUOM'))
        InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()

Preps.reset_index(drop=True, inplace=True)

In [19]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-55516,BAKED|Lasagna|Spin Mushroom,5.550,Kg,
1,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,
2,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,
3,P-56502,BAKED|Pasta|Shrimp Pesto,5.760,Kg,
4,P-56433,BATCH|Shrimp Remoulade,1.600,Kg,


In [20]:
Preps.shape

(742, 5)

In [21]:
Preps.dtypes

PrepId            object
Description       object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [22]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

#### Load Product List

In [23]:
ProdId = []
Description = []

for filepath in filepath_list:
    path = filepath + '/Products.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for x in xtree.iterfind('Prod'):
        ProdId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description}).drop_duplicates()

Products.reset_index(drop=True, inplace=True)

In [24]:
Products.head()

,ProdId,Description
0,R-61778,ALF|Flatbread|4 Cheese
1,R-61780,ALF|Flatbread|Apple & Pancetta
2,R-61749,ALF|Flatbread|BBQ Chicken
3,R-50859,ALF|Flatbread|Bruschetta
4,R-50788,ALF|Flatbread|Caprese


In [25]:
Products.shape

(441, 2)

In [26]:
Products.dtypes

ProdId         object
Description    object
dtype: object

In [27]:
path = os.path.join(os.getcwd(), "data", "processed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

#### Load Conversion List

In [28]:
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

for filepath in filepath_list:
    path = filepath + '/Conversions.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for x in xtree.iterfind('Conversion'):
        ConversionId.append(x.attrib['id'])
        Multiplier.append(x.attrib['multiplier'])
        ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
        ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
        ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
        ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [29]:
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L


In [30]:
Conversions.shape

(294, 6)

In [31]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [32]:
path = os.path.join(os.getcwd(), "data", "processed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

#### Load Cool Food Calculator Emission Factors

In [33]:
cfc_filename = os.path.join(os.getcwd(), "data", "raw", "cool_food_ghg.csv")
cfc = pd.read_csv(cfc_filename)

In [34]:
cfc.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [35]:
cfc.dtypes

Category ID                                                 int64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [36]:
cfc.shape

(61, 3)

#### Standard Unit Converter

In [37]:
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "raw", "standard_conversions.csv"))
Std_Unit#.head()

,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,4.928900,1,tsp,4.928900,ml
1,14.787000,1,Tbsp,14.787000,ml
2,946.350000,1,qt,946.350000,ml
3,473.176250,1,pt,473.176250,ml
4,28.349500,1,oz,28.349500,g
5,0.001000,1000,mg,1.000000,g
6,453.592000,1,lb,453.592000,g
7,1000.000000,1,L,1000.000000,ml
8,1000.000000,1,Kg,1000.000000,g
9,58600.000000,1,keg59L,58600.000000,ml


In [38]:
liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
liquid_unit

['tsp',
 'Tbsp',
 'qt',
 'pt',
 'L',
 'keg59L',
 'keg50L',
 'KEG 59L',
 'KEG 50L',
 'KEG 30L',
 'gal',
 'fl oz',
 'cup',
 '750ML',
 '710ML',
 '700ML',
 '660ML',
 '650ML',
 '591ML',
 '500ML',
 '4L',
 '473ML',
 '450ML',
 '355ML',
 '2L',
 '16L',
 '1140ml',
 '1/2LTR',
 '1.75 L',
 '1.5L',
 '1.14L']

In [39]:
solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()
solid_unit

['oz', 'mg', 'lb', 'Kg']

In [40]:
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        Qty = float(qty)*float(multiplier)
        Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
    else:
        Qty = qty
        Uom = uom
    return (Qty, Uom)

In [41]:
std_converter(0.25,'lb')

(113.398, 'g')

In [42]:
spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))

In [43]:
def spc_converter(ingre, qty, uom):
    if uom in liquid_unit + solid_unit:
        return std_converter(qty, uom)
    elif ingre in spc_cov:
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
                                    & (Conversions['ConvertToUom'] == 'g')]
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else: 
            Qty = float(qty)/float(multiplier)
            Uom = conversion['ConvertToUom'].values[0]
            return (Qty, Uom)
    else:
        return std_converter(qty, uom)

In [44]:
spc_converter('I-1120', 1, 'CT')

(50.0, 'g')

## Data Cleaning

#### Unit Conversion

In [45]:
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

In [46]:
for index in Preps.index:
    PrepId = Preps.loc[index,'PrepId']
    Qty = Preps.loc[index,'PakQty']
    Uom = Preps.loc[index,'PakUOM']
    Preps.loc[index,'StdQty'] = spc_converter(PrepId, Qty, Uom)[0]
    Preps.loc[index,'StdUom'] = spc_converter(PrepId, Qty, Uom)[1]

In [47]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-55516,BAKED|Lasagna|Spin Mushroom,5.550,Kg,,5550,g
1,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,,6176,g
2,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,,7360,g
3,P-56502,BAKED|Pasta|Shrimp Pesto,5.760,Kg,,5760,g
4,P-56433,BATCH|Shrimp Remoulade,1.600,Kg,,1600,g
5,P-50500,CHIFFONADE|Basil,190.000,g,,190.000,g
6,P-24750,CHOPPED|Cilantro,0.500,Kg,,500,g
7,P-53735,CHOPPED|pickle,900.000,g,,900.000,g
8,P-58450,COOKED|Bahn Mi Chicken,8.500,Kg,,8500,g
9,P-26068,COOKED|Caramelized Onion,1.200,Kg,PREP,1200,g


#### Get Preps with Nonstandard Unit

In [48]:
col_names = list(Preps.columns.values)
Preps_Nonstd = []

for index, row in Preps.iterrows():
    StdUom = Preps.loc[index,'StdUom']
    if StdUom not in ['g', 'ml']:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd, columns = col_names)

In [49]:
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.000,each,PREP,8.000,each
1,P-35132,MARINATED|Lemon & Herb Chx,185.000,ea,PREP,185.000,ea
2,P-51992,YIELD|Bread|Sourdough 5/8,36.000,slice,,36.000,slice
3,P-26234,BATCH|Roasted Garlic Bread,16.000,ea,PREP,16.000,ea
4,P-26170,GRILLED|NaanBread,1.000,ea,PREP,1.000,ea
5,P-16305,YIELD|Smokie (1pc),1.000,ea,,1.000,ea
6,P-26047,BOILED|Hard Boiled Eggs FT,50.000,ea,PREP,50.000,ea
7,P-26631,BRK|Pancake|Chocolate Chip,24.000,ea,,24.000,ea
8,P-26057,FRIED|Sunny/Overeasy Egg,1.000,ea,ZDONT USE OK - PREP,1.000,ea
9,P-26056,GRILLED|Tomato,1.000,ea,ZDONT USE OK - PREP,1.000,ea


In [50]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_NonstdUom.csv")
Preps_Nonstd.to_csv(path, index = False, header = True)

#### Update Correct Uom for Preps

In [51]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

## Data Analysis

#### Add New Items to Database

In [52]:
# Load current Items List with assigned Cool Food Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "processed", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT
2,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
3,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
4,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT


In [53]:
Items_Assigned.shape

(781, 8)

In [54]:
col_names = list(Items.columns.values)

In [55]:
New_Items_List = []

for index, row in Items.iterrows():
    ItemId = Items.loc[index,'ItemId']
    if ItemId not in Items_Assigned['ItemId'].values:
        Dict = {}
        Dict.update(dict(row))
        New_Items_List.append(Dict)

New_Items = pd.DataFrame(New_Items_List, columns = col_names)

In [56]:
#Items_Assigned['ItemId'].values

In [57]:
New_Items.insert(1, "CategoryID", '')

In [58]:
New_Items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [59]:
New_Items.shape

(0, 8)

In [60]:
from datetime import datetime
path = os.path.join(os.getcwd(), "data", "new items", str(datetime.date(datetime.now()))+"_New_Items.csv")
New_Items.to_csv(path, index = False, header = True)

In [61]:
New_Items_Added = pd.read_csv(os.path.join(os.getcwd(), "data", "new items added", "New_Items_Added_4.csv"))
New_Items_Added

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-4642,40,EGGPLANT JAPANESEMX,30.000,lb,1.00,lb,PRODUCE
1,I-4792,40,PEPPERS RED CHOICE US,25.000,lb,1.00,lb,PRODUCE
2,I-2658,59,PEROGY POT CHED CKD 180CT-30G,2.000,bag,2.75,Kg,FOOD - GROCERY
3,I-2319,3,SMOKIES 5.5 CT BAVARIAN,1.000,Kg,1.00,Kg,MEAT
4,I-3617,54,ALLSPICE GROUND SHAKER,8.000,each,454.00,g,SPICES
5,I-3838,1,"BEEF BRISKET BNLS 3/4"" AAA",1.000,Kg,1.00,Kg,MEAT
6,I-4523,38,"BEETS DICED 3/4 BC""",5.000,lb,1.00,lb,PRODUCE
7,I-3992,4,CHICK BACKS & NECKS -FRESH,1.000,Kg,1.00,Kg,POULTRY
8,I-3633,54,GINGER GROUND SHAKER TFC,1.000,each,454.00,g,SPICES
9,I-11172,54,PEPPER BLK WHOLE,2.000,Kg,1.00,Kg,SPICES


In [62]:
frames = [Items_Assigned, New_Items_Added]
Items_Assigned_Updated = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()
Items_Assigned_Updated.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT
2,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
3,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
4,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT


In [63]:
Items_Assigned_Updated.shape

(781, 8)

In [64]:
Items_Assigned_Updated[['CategoryID']] = Items_Assigned_Updated[['CategoryID']].apply(pd.to_numeric)

In [65]:
path = os.path.join(os.getcwd(), "data", "processed", "Items_List_Assigned.csv")
Items_Assigned_Updated.to_csv(path, index = False, header = True)

#### Mapping Items to GHG Factors

In [66]:
df1 = pd.DataFrame(Items_Assigned_Updated)
df2 = pd.DataFrame(cfc)

In [67]:
df1.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT
2,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
3,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
4,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT


In [68]:
df1.dtypes

ItemId             object
CategoryID        float64
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [69]:
df2.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [70]:
df2.dtypes

Category ID                                                 int64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [71]:
mapping = pd.merge(df1, df2.loc[:,['Category ID','Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

In [72]:
for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = 0

##### Manully Adjust GHG Factor for Some Items

In [73]:
Manual_GHG = pd.read_csv(os.path.join(os.getcwd(), "data", "processed", "Manual_Adjust_GHGe.csv"))
Manual_GHG

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-3223,59,COCONUT MILK 17/19% MILK FAT,6.00,LG CAN,2.84,L,FOOD - GROCERY,59,manually adjusted,3.50000
1,I-3370,59,MUSTARD DIJON HOT,2.00,each,4625.00,ml,FOOD - GROCERY,59,manually adjusted,2.32600
2,I-3368,59,MUSTARD DIJON GRAINY OLD FASH,2.00,pail,4625.00,ml,FOOD - GROCERY,59,manually adjusted,2.32600
3,I-2897,59,MUSTARD DIJON GRAINY FLEUR,4.00,Kg,1.00,Kg,FOOD - GROCERY,59,manually adjusted,2.32600
4,I-3365,59,MUSTARD BIG YELLOW PLAS,4.00,JUG,2.84,L,FOOD - GROCERY,59,manually adjusted,1.76000
5,I-3352,59,MAYONNAISE PAIL TFC 4L,2.00,each,4.00,L,FOOD - GROCERY,59,manually adjusted,3.55000
6,I-3348,59,MAYONNAISE 1/2 FAT TFC,2.00,4L,4.00,L,FOOD - GROCERY,59,manually adjusted,1.22000
7,I-3336,59,KETCHUP BIG RED PLAS,6.00,LG CAN,2.84,L,FOOD - GROCERY,59,manually adjusted,2.01000
8,I-3284,59,EXTRACT VANILLA PURE,1.00,500ML,1.00,500ML,FOOD - GROCERY,59,manually adjusted,0.00000
9,I-52090,1,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.00,CT,MEAT,1,beef & buffalo meat,31.13750


In [74]:
for index, row in Manual_GHG.iterrows():
    itemId = Manual_GHG.loc[index, 'ItemId']
    ghge = Manual_GHG.loc[index, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
    mapping.loc[mapping['ItemId'] == itemId, 'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = ghge

In [75]:
mapping#.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.300,Kg,1.000,Kg,MEAT,1.0,beef & buffalo meat,41.34630
1,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.000,cs,48.000,CT,MEAT,1.0,beef & buffalo meat,31.13750
2,I-10869,1.0,BEEF STIRFRY COV FR,5.000,Kg,1.000,Kg,MEAT,1.0,beef & buffalo meat,41.34630
3,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.000,Kg,1.000,Kg,MEAT,1.0,beef & buffalo meat,41.34630
4,I-37005,1.0,BEEF MEATBALLS,4.540,Kg,1000.000,g,MEAT,1.0,beef & buffalo meat,41.34630
5,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.000,Kg,1000.000,g,MEAT,1.0,beef & buffalo meat,41.34630
6,I-3876,1.0,BEEF CHUCK GROUND AAA,1.000,Kg,1.000,Kg,MEAT,1.0,beef & buffalo meat,41.34630
7,I-13303,3.0,SAUSAGE PATTIES PORK,5.000,Kg,20.000,ea,MEAT,3.0,pork (pig meat),9.83150
8,I-11922,3.0,SAUSAGE MEAT CHORIZO FRZN,5.000,CT,2.000,Kg,MEAT,3.0,pork (pig meat),9.83150
9,I-50575,3.0,SAUSAGE ITAL CRUMBLE CKD FRZ,2.000,bag,2.270,Kg,MEAT,3.0,pork (pig meat),9.83150


In [76]:
mapping.shape

(781, 11)

In [77]:
mapping.dtypes

ItemId                                                     object
CategoryID                                                float64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Category ID                                               float64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [78]:
path = os.path.join(os.getcwd(), "data", "processed", "Mapping.csv")
mapping.to_csv(path, index = False, header = True)

#### GHG Factors Calculation for Preps

In [79]:
Preps['GHG Emission (g)'] = 0
Preps['GHG Emission(g)/StdUom'] = 0

In [80]:
def get_items_ghge_prep(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghg = Preps.loc[index, 'GHG Emission (g)']
    weight = Preps.loc[index, 'StdQty']
    print('Index:', index, '\nIngres:\n', ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            #print(ghge)
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in spc_cov:
                qty = spc_converter(ingre, Qty, Uom)[0]
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                ghg += qty*float(ghge)
            print(ingre, Qty, Uom, qty, float(ghge), qty*float(ghge))
    Preps.loc[index, 'GHG Emission (g)'] = float(ghg)
    Preps.loc[index, 'GHG Emission(g)/StdUom'] = ghg/float(weight)

In [81]:
def get_preps_ghge_prep(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghg = Preps.loc[index, 'GHG Emission (g)']
    weight = Preps.loc[index, 'StdQty']
    print('Index:', index, '\nIngres:\n', ingres)
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('P'):
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission(g)/StdUom']
            #print(ghge)
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in spc_cov:
                qty = spc_converter(ingre, Qty, Uom)[0]
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                ghg += qty*float(ghge)
            print(ingre, Qty, Uom, qty, qty*float(ghge))
        #print('Final:\n', ghg, weight)
    Preps.loc[index, 'GHG Emission (g)'] = float(ghg)
    Preps.loc[index, 'GHG Emission(g)/StdUom'] = ghg/float(weight)

In [82]:
def link_preps(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    ghg = Preps.loc[index, 'GHG Emission (g)']
    weight = Preps.loc[index, 'StdQty']
    if len(ingres) == 1:
        ingre = ingres.iloc[0]['IngredientId']
        if ingre.startswith('P'):
            print(ingres)
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission(g)/StdUom']
            Qty = float(ingres.iloc[0]['Qty'])
            Uom = ingres.iloc[0]['Uom']
            if ingre in spc_cov:
                qty = spc_converter(ingre, Qty, Uom)[0]
                ghg = qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                ghg = qty*float(ghge)
            print(ingre, ghge, Qty, Uom, qty, weight)
            print(ghg/float(weight))
    Preps.loc[index, 'GHG Emission (g)'] = float(ghg)
    Preps.loc[index, 'GHG Emission(g)/StdUom'] = ghg/float(weight)

In [83]:
for index, row in Preps.iterrows():
    get_items_ghge_prep(index , row)

Index: 0 
Ingres:
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
272      I-14179  500.000   g  0.00100000   90.0901  P-55516
273       I-2141  100.000   g  0.00100000   18.0180  P-55516
274       I-2146  500.000   g  0.00100000   90.0901  P-55516
275       I-3408  600.000   g  0.00220462  108.1081  P-55516
276      P-26076    2.250   L  1.00000000    0.4054  P-55516
277      P-55515    1.600  Kg  1.00000000    0.2883  P-55516
I-14179 500.0 g 500.0 8.9104 4455.2
I-2141 100.0 g 100.0 8.9104 891.04
I-2146 500.0 g 500.0 8.9104 4455.2
I-3408 600.0 g 600.0 1.5225 913.5
Index: 1 
Ingres:
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
242      I-14179  720.000   g  0.00100000  116.5803  P-54666
243       I-2141  160.000   g  0.00100000   25.9067  P-54666
244      I-41921  800.000   g  0.00100000  129.5337  P-54666
245      P-26216    2.400  Kg  1.00000000    0.3886  P-54666
246      P-34121  500.000   g  0.00100000   80.9585  P-54666
247       P-8196    1.600   L

Index: 36 
Ingres:
     IngredientId    Qty Uom  Conversion InvFactor   Recipe
248       I-6876  0.250  lb  1.00000000    0.0313  P-54697
I-6876 0.25 lb 113.398 0.3942 44.7014916
Index: 37 
Ingres:
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
299       I-3658   30.000   g  0.00100000   17.6471  P-56490
300       I-4007    1.600  Kg  1.00000000    0.9412  P-56490
301      I-45114  250.000  ml  0.03381406  147.0588  P-56490
302       I-4796   10.000   g  0.00220462    5.8824  P-56490
303       I-6443  250.000  ml  0.03381406  147.0588  P-56490
304      P-26020   20.000   g  0.00100000   11.7647  P-56490
I-3658 30.0 g 30.0 1.6414 49.242
I-4007 1.6 Kg 1600.0 4.3996 7039.360000000001
I-45114 250.0 ml 250.0 0.0 0.0
I-4796 10.0 g 10.0 0.5029 5.029
I-6443 250.0 ml 250.0 0.3942 98.55
Index: 38 
Ingres:
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
119       I-3388    2.000   L  1.00000000    0.0108  P-35132
120       I-4031   23.000  Kg  1.00000000    0.1243  P-

Index: 59 
Ingres:
     IngredientId    Qty Uom     Conversion InvFactor   Recipe
159      I-35791  3.000  Kg  1000.00000000    1.1111  P-46862
I-35791 3.0 Kg 3000.0 0.6932 2079.6
Index: 60 
Ingres:
     IngredientId    Qty Uom  Conversion InvFactor   Recipe
349      P-19175  1.000  Kg  1.00000000    1.0000  P-57639
Index: 61 
Ingres:
    IngredientId      Qty Uom  Conversion InvFactor   Recipe
23       I-3388  750.000  ml  0.00100000  300.0000  P-26020
24       I-4657    2.270  Kg  2.20462000    0.9080  P-26020
I-3388 750.0 ml 750.0 3.2401 2430.075
I-4657 2.27 Kg 2270.0 0.3015 684.405
Index: 62 
Ingres:
   IngredientId    Qty Uom  Conversion InvFactor   Recipe
1       I-3388  1.000   L  1.00000000    0.3058  P-10496
2       I-4660  2.270  Kg  2.20462000    0.6942  P-10496
I-3388 1.0 L 1000.0 3.2401 3240.1
I-4660 2.27 Kg 2270.0 0.3062 695.0740000000001
Index: 63 
Ingres:
     IngredientId     Qty Uom  Conversion InvFactor   Recipe
331       I-1813  60.000  ml  0.03381406   66.6667  P-5

Index: 78 
Ingres:
    IngredientId      Qty    Uom  Conversion InvFactor   Recipe
52      I-11215    1.000      g  0.00220462    0.1990  P-26076
53      I-14179  285.000      g  0.00100000   56.7164  P-26076
54      I-17615   20.000  slice  1.00000000    3.9801  P-26076
55       I-2141  285.000      g  0.00100000   56.7164  P-26076
56       I-2251    4.000      L  1.00000000    0.7960  P-26076
57       I-3291  225.000      g  0.00100000   44.7761  P-26076
58       I-3619    1.000      g  1.00000000    0.1990  P-26076
59       I-4745  225.000      g  0.00220462   44.7761  P-26076
60       I-8060  225.000      g  1.00000000   44.7761  P-26076
61      P-18907    5.000      g  1.00000000    0.9950  P-26076
I-11215 1.0 g 1.0 9.3703 9.3703
I-14179 285.0 g 285.0 8.9104 2539.464
I-17615 20.0 slice 20.0 8.9104 178.20799999999997
I-2141 285.0 g 285.0 8.9104 2539.464
I-2251 4.0 L 4000.0 2.2325 8930.0
I-3291 225.0 g 225.0 1.5225 342.5625
I-3619 1.0 g 1.0 9.3703 9.3703
I-4745 225.0 g 225.0 0.3015 

I-4341 90.0 g 90.0 0.5029 45.261
Index: 107 
Ingres:
    IngredientId    Qty Uom  Conversion InvFactor   Recipe
97       I-4341  1.000  CT  1.00000000    0.0182  P-26514
I-4341 1.0 CT 1.0 0.5029 0.5029
Index: 108 
Ingres:
   IngredientId     Qty Uom  Conversion InvFactor   Recipe
9       I-4626  10.000  CT  1.00000000    2.6316  P-21077
I-4626 10.0 CT 3799.9984800006077 0.5029 1911.0192355923057
Index: 109 
Ingres:
     IngredientId    Qty Uom  Conversion InvFactor   Recipe
226       I-9832  5.000  Kg  2.20462000    1.1111  P-51222
I-9832 5.0 Kg 5000.0 0.3062 1531.0000000000002
Index: 110 
Ingres:
     IngredientId    Qty Uom  Conversion InvFactor   Recipe
342       I-4637  1.000  Kg  2.20462000    1.1111  P-57150
I-4637 1.0 Kg 1000.0 0.5029 502.90000000000003
Index: 111 
Ingres:
     IngredientId     Qty Uom  Conversion InvFactor   Recipe
149       I-3454  19.000   L  1.00000000    1.6667  P-44652
I-3454 19.0 L 19000.0 0.5029 9555.1
Index: 112 
Ingres:
     IngredientId    Qty Uom  Co

...

I-3619 5.0 g 5.0 9.3703 46.8515
I-3658 750.0 g 750.0 1.6414 1231.05
I-3698 375.0 ml 375.0 1.93 723.75
I-47774 1.8 L 1800.0 0.0 0.0
I-4796 25.0 g 25.0 0.5029 12.5725
Index: 645 
Ingres:
      IngredientId      Qty Uom  Conversion InvFactor   Recipe
3969       I-2217    1.000   L  1.00000000    0.1905  P-35898
3970       I-2254    2.000  ml  0.00100000    0.3810  P-35898
3971       I-3348    4.000   L  1.00000000    0.7619  P-35898
3972      I-37480  425.000   g  1.00000000   80.9524  P-35898
3973       I-6026    5.000   g  1.00000000    0.9524  P-35898
3974       I-6443  100.000  ml  0.03381406   19.0476  P-35898
I-2217 1.0 L 1000.0 6.9824 6982.400000000001
I-2254 2.0 ml 2.0 6.9824 13.9648
I-3348 4.0 L 4000.0 1.22 4880.0
I-37480 425.0 g 425.0 0.0 0.0
I-6026 5.0 g 5.0 0.44 2.2
I-6443 100.0 ml 100.0 0.3942 39.42
Index: 646 
Ingres:
      IngredientId      Qty Uom     Conversion InvFactor  Recipe
4239       I-2254  250.000  ml     0.00100000  125.0000  P-7136
4240       I-3387   50.000  ml

I-3695 100.0 ml 100.0 1.93 193.0
I-4657 100.0 g 100.0 0.3015 30.15
I-57545 10.0 Kg 10000.0 41.3463 413463.0
Index: 663 
Ingres:
      IngredientId      Qty Uom  Conversion InvFactor   Recipe
4163      I-11312   40.000   g  0.03527399    2.9197  P-56816
4164      I-11661   10.000  Kg  1.00000000    0.7299  P-56816
4165       I-3223    7.000   L  1.00000000    0.5109  P-56816
4166       I-3627   30.000   g  0.03527399    2.1898  P-56816
4167       I-3654  300.000   g  1.00000000   21.8978  P-56816
4168       I-4655  800.000   g  0.00220462   58.3942  P-56816
4169       I-4657  400.000   g  0.00220462   29.1971  P-56816
4170       I-4796  220.000   g  0.00220462   16.0584  P-56816
4171       I-4894    3.000  Kg  2.20462000    0.2190  P-56816
4172       I-9685  800.000   g  0.00220462   58.3942  P-56816
4173      P-56819  600.000   g  1.00000000   43.7956  P-56816
I-11312 40.0 g 40.0 0.3942 15.768
I-11661 10.0 Kg 10000.0 41.3463 413463.0
I-3223 7.0 L 7000.0 3.5 24500.0
I-3627 30.0 g 30.0 9

4532      P-18907   45.000   g  1.00000000   10.7143  P-55115
I-19737 4.0 Kg 4000.0 0.6932 2772.8
I-2911 5.0 g 5.0 9.3703 46.8515
I-3388 100.0 ml 100.0 3.2401 324.01
I-3623 5.0 g 5.0 9.3703 46.8515
I-3630 5.0 g 5.0 4.2854 21.427
I-4657 65.0 g 65.0 0.3015 19.5975
I-4745 200.0 g 200.0 0.3015 60.3
I-4787 15.0 g 15.0 0.5029 7.5435
Index: 700 
Ingres:
      IngredientId    Qty Uom  Conversion InvFactor   Recipe
4472       I-1813  5.000  ml  0.03381406    1.2500  P-44506
4473       I-4472  1.000  CT  1.00000000    0.2500  P-44506
I-1813 5.0 ml 5.0 0.3942 1.971
I-4472 1.0 CT 136.00002176000348 0.4306 58.56160936985749
Index: 701 
Ingres:
      IngredientId      Qty Uom   Conversion InvFactor   Recipe
4500       I-1813    1.000   L  35.21126761    0.0769  P-53339
4501       I-3233    5.000  Kg   1.00000000    0.3846  P-53339
4502       I-3388    1.000   L   1.00000000    0.0769  P-53339
4503       I-3625   12.000   g   1.00000000    0.9231  P-53339
4504       I-3628    2.000   g   1.00000000  

Index: 718 
Ingres:
      IngredientId      Qty Uom   Conversion InvFactor   Recipe
4877       I-1813    2.300   L  35.21126761    0.1983  P-57958
4878       I-3391  300.000  ml   0.00100000   25.8621  P-57958
4879       I-3590  240.000   g   0.00100000   20.6897  P-57958
4880      I-47774    1.200   L   1.00000000    0.1034  P-57958
4881       I-6028    1.800  Kg   1.00000000    0.1552  P-57958
4882      P-18907   15.000   g   1.00000000    1.2931  P-57958
4883      P-47541  750.000  ml   0.00100000   64.6552  P-57958
4884      P-53277    5.000   L   1.00000000    0.4310  P-57958
I-1813 2.3 L 2300.0 0.3942 906.66
I-3391 300.0 ml 300.0 3.1509 945.27
I-3590 240.0 g 240.0 4.2854 1028.496
I-47774 1.2 L 1200.0 0.0 0.0
I-6028 1.8 Kg 1800.0 1.6414 2954.52
Index: 719 
Ingres:
      IngredientId      Qty Uom  Conversion InvFactor   Recipe
4773      I-16215  169.000   g  0.00100000   42.2500  P-30552
4774      I-30504   22.000   g  1.00000000    5.5000  P-30552
4775       I-3388   30.000  ml  0

Index: 733 
Ingres:
      IngredientId      Qty Uom     Conversion InvFactor   Recipe
5044       I-3429    1.440   L  1000.00000000    0.3600  P-42292
5045       I-6026   20.000   g     1.00000000    5.0000  P-42292
5046       I-6850    4.000  Kg  1000.00000000    1.0000  P-42292
5047      P-24750  100.000   g     0.00100000   25.0000  P-42292
5048      P-34880  100.000   g     0.00100000   25.0000  P-42292
I-3429 1.44 L 1440.0 0.0 0.0
I-6026 20.0 g 20.0 0.44 8.8
I-6850 4.0 Kg 4000.0 2.9782 11912.800000000001
Index: 734 
Ingres:
      IngredientId      Qty Uom     Conversion InvFactor   Recipe
5094       I-2852   10.000   g     1.00000000    3.5714  P-61678
5095       I-2911    5.000   g     0.00100000    1.7857  P-61678
5096       I-3390   60.000  ml     0.00100000   21.4286  P-61678
5097       I-3447  200.000  ml     1.00000000   71.4286  P-61678
5098       I-3630   10.000   g     0.03527399    3.5714  P-61678
5099       I-3695   20.000  ml     0.00100000    7.1429  P-61678
5100     

In [84]:
for index, row in Preps.iterrows():
    link_preps(index, row)

   IngredientId      Qty Uom  Conversion  InvFactor   Recipe
12      P-26514  500.000   g  1.00000000  1000.0000  P-24750
P-26514 107    0.009144
Name: GHG Emission(g)/StdUom, dtype: float64 500.0 g 500.0 500.0
0.009143636363636364
    IngredientId     Qty Uom  Conversion InvFactor   Recipe
351      P-56490  10.000  Kg  1.00000000    1.1765  P-58450
P-56490 37    4.230695
Name: GHG Emission(g)/StdUom, dtype: float64 10.0 Kg 10000.0 8500.0
4.977287889273358
    IngredientId     Qty Uom  Conversion InvFactor   Recipe
146      P-19175  11.500  Kg  1.00000000    1.0445  P-43440
P-19175 123    0.427753
Name: GHG Emission(g)/StdUom, dtype: float64 11.5 Kg 11500.0 11010.0
0.4467902758855586
    IngredientId     Qty Uom  Conversion InvFactor   Recipe
348      P-19175  11.500  Kg  1.00000000    1.0455  P-57499
P-19175 123    0.427753
Name: GHG Emission(g)/StdUom, dtype: float64 11.5 Kg 11500.0 11000.0
0.44719644886363635
    IngredientId    Qty Uom  Conversion InvFactor  Recipe
397      P-18746

     IngredientId     Qty Uom  Conversion InvFactor   Recipe
3633      P-46076  20.000   g  0.00100000    1.0000  P-49552
P-46076 183    28.09
Name: GHG Emission(g)/StdUom, dtype: float64 20.0 g 20.0 20.000
28.089999999999996
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
4100      P-50692  100.000   g  1.00000000    0.7692  P-50691
P-50692 229    1.85154
Name: GHG Emission(g)/StdUom, dtype: float64 100.0 g 100.0 130.000
1.4242615384615385


In [85]:
for index, row in Preps.iterrows():
    get_preps_ghge_prep(index, row)

Index: 0 
Ingres:
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
272      I-14179  500.000   g  0.00100000   90.0901  P-55516
273       I-2141  100.000   g  0.00100000   18.0180  P-55516
274       I-2146  500.000   g  0.00100000   90.0901  P-55516
275       I-3408  600.000   g  0.00220462  108.1081  P-55516
276      P-26076    2.250   L  1.00000000    0.4054  P-55516
277      P-55515    1.600  Kg  1.00000000    0.2883  P-55516
P-26076 2.25 L 2250.0 7696.292507462687
P-55515 1.6 Kg 1600.0 480.03478260869565
Index: 1 
Ingres:
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
242      I-14179  720.000   g  0.00100000  116.5803  P-54666
243       I-2141  160.000   g  0.00100000   25.9067  P-54666
244      I-41921  800.000   g  0.00100000  129.5337  P-54666
245      P-26216    2.400  Kg  1.00000000    0.3886  P-54666
246      P-34121  500.000   g  0.00100000   80.9585  P-54666
247       P-8196    1.600   L  1.00000000    0.2591  P-54666
P-26216 2.4 Kg 2400.0 2161.

Index: 34 
Ingres:
     IngredientId     Qty Uom  Conversion InvFactor   Recipe
171      P-50429  90.000   g  1.00000000    1.0000  P-47377
P-50429 90.0 g 90.0 70.406
Index: 35 
Ingres:
     IngredientId     Qty Uom  Conversion InvFactor   Recipe
227       I-4572  11.450  Kg  2.20462000    1.0905  P-51582
Index: 36 
Ingres:
     IngredientId    Qty Uom  Conversion InvFactor   Recipe
248       I-6876  0.250  lb  1.00000000    0.0313  P-54697
Index: 37 
Ingres:
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
299       I-3658   30.000   g  0.00100000   17.6471  P-56490
300       I-4007    1.600  Kg  1.00000000    0.9412  P-56490
301      I-45114  250.000  ml  0.03381406  147.0588  P-56490
302       I-4796   10.000   g  0.00220462    5.8824  P-56490
303       I-6443  250.000  ml  0.03381406  147.0588  P-56490
304      P-26020   20.000   g  0.00100000   11.7647  P-56490
P-26020 20.0 g 20.0 24.915839999999996
Index: 38 
Ingres:
     IngredientId      Qty Uom  Conversion InvFact

2       I-4660  2.270  Kg  2.20462000    0.6942  P-10496
Index: 63 
Ingres:
     IngredientId     Qty Uom  Conversion InvFactor   Recipe
331       I-1813  60.000  ml  0.03381406   66.6667  P-56506
332       I-1820  60.000  ml  0.00100000   66.6667  P-56506
333       I-2911   2.000   g  0.00100000    2.2222  P-56506
334       I-3389  60.000  ml  0.00100000   66.6667  P-56506
335       I-3630   5.000   g  0.03527399    5.5556  P-56506
336       I-3640   5.000   g  1.00000000    5.5556  P-56506
337       I-3856   1.000  Kg  1.00000000    1.1111  P-56506
338       I-6028  50.000   g  0.00100000   55.5556  P-56506
339      P-26020  20.000   g  0.00100000   22.2222  P-56506
P-26020 20.0 g 20.0 24.915839999999996
Index: 64 
Ingres:
     IngredientId     Qty Uom  Conversion InvFactor   Recipe
128       I-4593   6.000  CT  1.00000000    1.3333  P-37780
129      P-18907  50.000   g  1.00000000   11.1111  P-37780
130       P-7139  88.720  ml  0.00100000   19.7156  P-37780
P-18907 50.0 g 50.0 104.

194      I-43648  4.000  Kg  1.00000000    1.0101  P-50488
Index: 98 
Ingres:
     IngredientId       Qty Uom  Conversion InvFactor   Recipe
203       I-5983  1000.000  ml  0.00100000  500.0000  P-50495
204       I-6820  1000.000   g  0.00100000  500.0000  P-50495
Index: 99 
Ingres:
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
104       I-3141  946.000  ml  1.00000000   30.4180  P-32661
105       I-5983   31.000   L  1.00000000    0.9968  P-32661
Index: 100 
Ingres:
     IngredientId      Qty Uom  Conversion InvFactor   Recipe
141       I-2254  900.000   g  0.00093884  900.0000  P-42012
142       I-4341    1.000   g  0.01259785    1.0000  P-42012
143       I-4712    1.000  CT  1.00000000    1.0000  P-42012
144       I-4795   45.000   g  0.00220462   45.0000  P-42012
145       I-6026    2.000   g  1.00000000    2.0000  P-42012
Index: 101 
Ingres:
     IngredientId     Qty Uom  Conversion InvFactor  Recipe
399      I-13050  13.000  lb  1.00000000    1.7370  P-9851
400   

599      P-26077  5.000   g  0.00100000    5.0000  P-26170
P-26077 5.0 g 5.0 47.09370417121981
Index: 146 
Ingres:
     IngredientId      Qty Uom     Conversion InvFactor   Recipe
629      I-11207  640.000   g     1.00000000   10.3226  P-34085
630      I-11216   80.000   g     0.00100000    1.2903  P-34085
631      I-11217  480.000  ml     0.00100000    7.7419  P-34085
632       I-1813  160.000  ml     0.03381406    2.5806  P-34085
633       I-3621  128.000   g     1.00000000    2.0645  P-34085
634       I-3630  320.000   g     0.03527399    5.1613  P-34085
635       I-4016   80.000  Kg     1.00000000    1.2903  P-34085
636       I-6850    8.000  Kg  1000.00000000    0.1290  P-34085
637      P-18907  640.000   g     1.00000000   10.3226  P-34085
638      P-26020  800.000   g     0.00100000   12.9032  P-34085
639      P-34880  800.000   g     0.00100000   12.9032  P-34085
P-18907 640.0 g 640.0 1331.3658775510205
P-26020 800.0 g 800.0 996.6335999999999
P-34880 800.0 g 800.0 255.166666666

...

Index: 627 
Ingres:
      IngredientId    Qty Uom  Conversion InvFactor   Recipe
4106       I-2254  1.000   L  1.00000000    1.0000  P-50826
4107       I-5983  0.250   L  1.00000000    0.2500  P-50826
Index: 628 
Ingres:
      IngredientId     Qty Uom  Conversion InvFactor   Recipe
4060       I-4593  12.000  CT  1.00000000    2.0000  P-44363
4061      I-51440  84.000   g  1.00000000   14.0000  P-44363
4062       I-6026  25.000   g  1.00000000    4.1667  P-44363
Index: 629 
Ingres:
      IngredientId      Qty Uom   Conversion InvFactor   Recipe
4233       I-3103  128.000  CT   1.00000000   32.0000  P-61496
4234      I-51440  254.000   g   1.00000000   63.5000  P-61496
4235      P-18907    4.000  oz  28.34950000    1.0000  P-61496
P-18907 4.0 oz 113.398 235.8972309102041
Index: 630 
Ingres:
      IngredientId      Qty Uom  Conversion InvFactor   Recipe
3975      I-10869    5.000  Kg  1.00000000    1.0000  P-36075
3976      P-18907   30.000   g  1.00000000    6.0000  P-36075
3977      P-2

P-18907 4.0 g 4.0 8.321036734693879
P-26020 20.0 g 20.0 24.915839999999996
Index: 648 
Ingres:
      IngredientId      Qty Uom     Conversion InvFactor   Recipe
4154       I-3391  300.000  ml     0.00100000   37.5000  P-56558
4155       I-3660  400.000   g     0.00100000   50.0000  P-56558
4156       I-3696    1.500   L  1000.00000000    0.1875  P-56558
4157      I-47774    3.000   L     1.00000000    0.3750  P-56558
4158       I-5983    3.000   L     1.00000000    0.3750  P-56558
Index: 649 
Ingres:
      IngredientId      Qty  Uom    Conversion InvFactor   Recipe
3931       I-3392  125.000   ml    0.00100000   41.6667  P-27492
3932       I-3696    1.000  cup  236.58800000    0.3333  P-27492
3933       I-6820   23.000    g    0.00100000    7.6667  P-27492
3934      P-26020  500.000    g    0.00100000  166.6667  P-27492
3935      P-34843    2.000    L    1.00000000    0.6667  P-27492
3936      P-34880  325.000    g    0.00100000  108.3333  P-27492
P-26020 500.0 g 500.0 622.895999999999

4417      P-56797  55.000   g  0.00100000   40.4115  P-26378
P-18907 10.0 g 10.0 20.802591836734695
P-26020 15.0 g 15.0 18.686879999999995
P-50500 15.0 g 15.0 7.940526315789474
P-56797 55.0 g 55.0 16.5825
Index: 674 
Ingres:
      IngredientId     Qty    Uom  Conversion InvFactor   Recipe
4516       I-4341   1.000     CT  1.00000000    0.0056  P-53594
4517       I-4724  25.000      g  0.00220462    0.1404  P-53594
4518       I-4772   1.000  BUNCH  1.00000000    0.0056  P-53594
Index: 675 
Ingres:
      IngredientId      Qty Uom  Conversion InvFactor   Recipe
4490       I-1813  500.000  ml  0.03381406   83.3333  P-50513
4491       I-3284   60.000  ml  0.00200000   10.0000  P-50513
4492       I-3660  650.000   g  0.00100000  108.3333  P-50513
4493       I-5012    5.000  Kg  1.00000000    0.8333  P-50513
4494       I-5983  375.000  ml  0.00100000   62.5000  P-50513
4495       I-6820   70.000   g  0.00100000   11.6667  P-50513
Index: 676 
Ingres:
      IngredientId      Qty Uom  Conversion

4473       I-4472  1.000  CT  1.00000000    0.2500  P-44506
Index: 701 
Ingres:
      IngredientId      Qty Uom   Conversion InvFactor   Recipe
4500       I-1813    1.000   L  35.21126761    0.0769  P-53339
4501       I-3233    5.000  Kg   1.00000000    0.3846  P-53339
4502       I-3388    1.000   L   1.00000000    0.0769  P-53339
4503       I-3625   12.000   g   1.00000000    0.9231  P-53339
4504       I-3628    2.000   g   1.00000000    0.1538  P-53339
4505       I-3630   40.000   g   0.03527399    3.0769  P-53339
4506       I-3640   20.000   g   1.00000000    1.5385  P-53339
4507       I-3654   20.000   g   1.00000000    1.5385  P-53339
4508       I-4832    1.600  Kg   2.20462000    0.1231  P-53339
4509      P-26311    5.500   L   1.00000000    0.4231  P-53339
4510      P-53594  800.000   g   1.00000000   61.5385  P-53339
P-26311 5.5 L 5500.0 776.655
P-53594 800.0 g 800.0 284.7883146067416
Index: 702 
Ingres:
      IngredientId     Qty Uom  Conversion InvFactor   Recipe
4434       I

Index: 725 
Ingres:
      IngredientId      Qty Uom  Conversion InvFactor   Recipe
4823      I-17593  275.000   g  0.00220462  275.0000  P-51825
4824      I-50575  150.000   g  0.00100000  150.0000  P-51825
4825       I-7388  150.000   g  0.00100000  150.0000  P-51825
4826      P-34121  150.000   g  0.00100000  150.0000  P-51825
4827      P-50372  275.000   g  0.00100000  275.0000  P-51825
P-34121 150.0 g 150.0 826.0951457908164
P-50372 275.0 g 275.0 3104.4203827932097
Index: 726 
Ingres:
      IngredientId     Qty Uom  Conversion InvFactor   Recipe
5057       I-4341   1.000  CT  1.00000000    0.0200  P-50428
5058       I-4724  25.000   g  0.00220462    0.5000  P-50428
Index: 727 
Ingres:
      IngredientId    Qty Uom     Conversion InvFactor   Recipe
5086      I-17055  2.270  Kg  1000.00000000    1.1350  P-57429
5087      P-56741  1.200  Kg     1.00000000    0.6000  P-57429
P-56741 1.2 Kg 1200.0 359.47760000000005
Index: 728 
Ingres:
      IngredientId      Qty Uom  Conversion InvFact

In [86]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom,GHG Emission (g),GHG Emission(g)/StdUom
0,P-55516,BAKED|Lasagna|Spin Mushroom,5.550,Kg,,5550,g,18891.267290,3.403832
1,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,,6176,g,17657.751270,2.859092
2,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,,7360,g,22177.046905,3.013186
3,P-56502,BAKED|Pasta|Shrimp Pesto,5.760,Kg,,5760,g,29040.084386,5.041681
4,P-56433,BATCH|Shrimp Remoulade,1.600,Kg,,1600,g,12750.771772,7.969232
5,P-50500,CHIFFONADE|Basil,190.000,g,,190.000,g,100.580000,0.529368
6,P-24750,CHOPPED|Cilantro,0.500,Kg,,500,g,9.143636,0.018287
7,P-53735,CHOPPED|pickle,900.000,g,,900.000,g,502.900000,0.558778
8,P-58450,COOKED|Bahn Mi Chicken,8.500,Kg,,8500,g,84613.894118,9.954576
9,P-26068,COOKED|Caramelized Onion,1.200,Kg,PREP,1200,g,3010.091750,2.508410


### GHG Calculation for Products

In [87]:
Products['GHG Emission (g)'] = np.nan
Products['Weight (g)'] = np.nan

In [88]:
for index, row in Products.iterrows():
    ghg = 0
    weight = 0
    Products.loc[index, 'GHG Emission (g)'] = ghg
    Products.loc[index, 'Weight (g)'] = ghg

In [89]:
ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[107,'ProdId']]
ingres

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
1487,P-47775,150.000,g,0.00100000,1.0000,R-25415
1488,P-50380,35.000,ml,0.00100000,1.0000,R-25415
1489,P-50651,90.000,g,0.00100000,1.0000,R-25415
1490,P-51582,20.000,g,0.00100000,1.0000,R-25415
1491,P-61660,20.000,g,1.00000000,1.0000,R-25415
1492,P-7736,120.000,g,0.00100000,1.0000,R-25415


In [90]:
def get_items_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    #print(ingres)
    ghg = Products.loc[index, 'GHG Emission (g)']
    weight = Products.loc[index, 'Weight (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in Conversions['ConversionId'].tolist():
                qty = spc_converter(ingre, Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
            #print(ingre, Qty, Uom, qty)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)

In [91]:
def get_preps_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    weight = Products.loc[index, 'Weight (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('P'):
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission(g)/StdUom']
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in Conversions['ConversionId'].tolist():
                qty = spc_converter(ingre, Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)[0]
                weight += qty
                ghg += qty*float(ghge)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)

In [92]:
def get_products_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    weight = Products.loc[index, 'Weight (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('R'):
            ghge = Products.loc[Products['ProdId'] == ingre, 'GHG Emission (g)']
            Weight = Products.loc[Products['ProdId'] == ingre, 'Weight (g)']
            Qty = float(ingres.loc[idx,'Qty'])
            ghg += Qty*float(ghge)
            weight += Qty*float(Weight)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)

In [93]:
for index, row in Products.iterrows():
    get_items_ghge(index , row)

In [94]:
for index, row in Products.iterrows():
    get_preps_ghge(index, row)

In [95]:
for index, row in Products.iterrows():
    get_products_ghge(index, row)

In [96]:
Products['GHG Emission (Kg) / 100g'] = 0.1*Products['GHG Emission (g)']/Products['Weight (g)']

In [97]:
Products

,ProdId,Description,GHG Emission (g),Weight (g),GHG Emission (Kg) / 100g
0,R-61778,ALF|Flatbread|4 Cheese,1229.947500,185.000000,0.664836
1,R-61780,ALF|Flatbread|Apple & Pancetta,756.345750,140.000000,0.540247
2,R-61749,ALF|Flatbread|BBQ Chicken,1023.874995,245.000000,0.417908
3,R-50859,ALF|Flatbread|Bruschetta,522.037267,215.000000,0.242808
4,R-50788,ALF|Flatbread|Caprese,629.237000,233.000000,0.270059
5,R-61781,ALF|Flatbread|Chick Artichoke,999.038200,245.000000,0.407771
6,R-50858,ALF|Flatbread|Greek,426.879716,200.000000,0.213440
7,R-61498,ALF|Flatbread|Hawaii Five-0,970.671530,200.000000,0.485336
8,R-50562,ALF|Flatbread|Italian,593.347237,141.000000,0.420814
9,R-56337,ALF|Flatbread|Mediterranean,313.698698,145.000000,0.216344


In [98]:
path = os.path.join(os.getcwd(), "data", "final", "Recipes GHG.csv")
Products.to_csv(path, index = False, header = True)